# Topic Modeling v2
https://github.com/shaoxiongji/sw-detection/blob/master/clf.py


In [17]:
import os
from pathlib import Path
import sys

In [2]:
project_name = 'clpsych'
project_path = Path(os.getcwd()).parent

if sys.platform == "win32":
    data_path = 'D:\Dataset\{0}\dataset'.format(project_name)
    model_path = 'D:\Dataset\{0}\models'.format(project_name)
    src_path = '/Volumes/Dataset/{0}/src'.format(project_name)
    
elif sys.platform == 'darwin':
    data_path = '/Volumes/Dataset/{0}/dataset'.format(project_name)
    model_path = '/Volumes/Dataset/{0}/models'.format(project_name)
    src_path = '/Volumes/Dataset/{0}/src'.format(project_name)
    
else:
    data_path = Path(project_path, 'dataset')
    model_path = Path(project_path, 'models')
    src_path = Path(project_path, 'src')

utils_path = str(Path(project_path, 'utils'))
# including the project folder and the utils folder
if utils_path not in ''.join(sys.path):
    sys.path.extend([str(project_path), utils_path, str(src_path)])

print('project path = {0}'.format(project_path))
print('data path = {0}'.format(data_path))
print('model path = {0}'.format(model_path))
print('sys.path = {0}'.format(sys.path))

project path = /home/guerramarj/github/clpsych
data path = /home/guerramarj/github/clpsych/dataset
model path = /home/guerramarj/github/clpsych/models
sys.path = ['/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages/vsc_install-0.11.3-py2.7.egg', '/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages/vsc_base-2.8.3-py2.7.egg', '/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages/easybuild_framework-3.8.1-py2.7.egg', '/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages/easybuild_easyblocks-3.8.1-py2.7.egg', '/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages/easybuild_easyconfigs-3.8.1-py2.7.egg', '', '/cm/shared/easybuild/software/EasyBuild/3.8.1/lib/python2.7/site-packages', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python36.zip', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/python3.6', '/home/guerramarj/packages/anaconda3/envs/deeplearning/lib/py

In [6]:
import string
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, utils, models, similarities
from collections import defaultdict
import pandas as pd
import string

In [4]:
data = pd.read_csv(Path(data_path, 'risk_title_body.csv'))
print(data.shape)
data.columns = ['index', 'risk_label', 'title_body']
data.drop(['index'], axis=1, inplace=True)
data.head()

(57015, 3)


,risk_label,title_body
0,a,"My first spoken word gig, contains rhythmic Co..."
1,a,My first spoken word gig. Warning: contains rh...
2,a,"Me, the boy who first showed me what love was ..."
3,a,Being Proud @ NYC Pride 2010
4,a,I wish this guy had been at my high school...


In [5]:
def cleaning(article):
    punctuation = set(string.punctuation)
    lemmatize = WordNetLemmatizer()
    one = " ".join([i for i in article.lower().split() if i not in stopwords])
    two = "".join(i for i in one if i not in punctuation)
    three = " ".join(lemmatize.lemmatize(i) for i in two.lower().split())
    return three

def pred_new(doc):
    one = cleaning(doc).split()
    two = dictionary.doc2bow(one)
    return two

In [8]:
print("Processing Topics features ...")
stopwords = set(nltk.corpus.stopwords.words('english'))
text = data['title_body'].map(cleaning)
text_list = list()
for t in text:
    temp = t.split()
    text_list.append([i for i in temp if i not in stopwords])

Processing Topics features ...


In [9]:
topic_num = 10
dictionary = corpora.Dictionary(text_list)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]
ldamodel = LdaModel(doc_term_matrix, num_topics=topic_num, id2word = dictionary, passes=50)


probs = list()  # list of probability vectors
for t in text:
    prob = ldamodel[(pred_new(t))]
    d = dict(prob)
    for i in range(topic_num):
        if i not in d.keys():
            d[i] = 0
    temp = list()
    for i in range(topic_num):
        temp.append(d[i])
    probs.append(temp)

In [16]:
topics = ldamodel.show_topics(num_topics=num_topics,formatted=False,num_words=20)
topics

[(0,
  [('person', 0.15955004),
   ('x', 0.035192695),
   ('2', 0.033360675),
   ('2url', 0.021206006),
   ('war', 0.015685301),
   ('3', 0.014806676),
   ('url', 0.009923845),
   ('bundleurl', 0.0094824005),
   ('star', 0.008823557),
   ('4', 0.008043063),
   ('old', 0.007956527),
   ('complete', 0.007714178),
   ('v', 0.0063779927),
   ('iiurl', 0.006050917),
   ('age', 0.0058938884),
   ('natural', 0.005420352),
   ('amp', 0.0052721673),
   ('tradestarterurl', 0.0046963873),
   ('starveurl', 0.004335181),
   ('empire', 0.0043012802)]),
 (1,
  [('arkham', 0.014341211),
   ('cat', 0.0074716066),
   ('la', 0.0063172407),
   ('packurl', 0.006292345),
   ('sniper', 0.006062058),
   ('hair', 0.005111568),
   ('collection', 0.005023238),
   ('seed', 0.0050196676),
   ('elite', 0.005018717),
   ('priority', 0.0048585394),
   ('terrarium', 0.004815473),
   ('skin', 0.0046895095),
   ('water', 0.0045282613),
   ('unity', 0.0045107105),
   ('color', 0.004242176),
   ('oxycodone', 0.0041780113)

In [13]:
coherence = ldamodel.top_topics(doc_term_matrix,dictionary=dictionary,topn=100)

In [15]:
topic2topkeywords = {}
topic2csb = {}
topic2keywords = {}
topic2csa = {}
num_topics =ldamodel.num_topics
cnt =1
for ws in coherence:
    wset = set(w[1] for w in ws[0])
    topic2topkeywords[cnt] = wset # set with top keywords for topic
    topic2csb[cnt] = ws[1] #avg coherence scores for each topic
    cnt +=1
for ws in topics:
    # create a unique set of keywords for each topic
    wset = set(w[0]for w in ws[1])
    topic2keywords[ws[0]+1] = wset
    
for i in range(1,num_topics+1):
    for j in range(1,num_topics+1):  
        if topic2keywords[i].intersection(topic2topkeywords[j])==topic2keywords[i]:
            topic2csa[i] = topic2csb[j]
finalData = pd.DataFrame([],columns=['Topic','words'])
finalData['Topic']=topic2keywords.keys()
finalData['Topic'] = finalData['Topic'].apply(lambda x: 'Topic'+str(x))
finalData['words']=topic2keywords.values()
finalData['cs'] = topic2csa.values()
finalData.sort_values(by='cs',ascending=False,inplace=True)
finalData

NameError: name 'topics' is not defined